In [1]:
%config InlineBackend.figure_formats = ['svg']
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import Model

import matplotlib.pyplot as plt
import matplotlib.gridspec as grid

import itertools
import time

# Structural Invariant and Equivariant Neural Networks

## Motivation
Strucutral approaches try to achieve invariance/equivariance 
**Definition**: Given a set $X:=\{x_1,x_2,\dots,x_n\}\in\Bbb{R}^{n\times d_q}$ with $x_i\in\Bbb{R}^{d_q}$, a set-based function is defined as:
                        $$f := \Bbb{R}^{n\times d_q}\rightarrow\Bbb{R}$$
                        
 <br> <br>                      
                        
**Properties**: A set-based function $f$ should be: 
    - permutation-invariant: 
$$f(X)= f(\pi(X))$$
    - permutation-equivariant:
$$\pi(f(X))= f(\pi(X))$$, s.t. $$\pi(X):= \{x_{\pi(1)},x_{\pi(2)},\dots,x_{\pi(n)}\}$$
    - process input sets of any size:
$$f := \Bbb{R}^{n\times d_q}\rightarrow\Bbb{R}, \forall n\in\Bbb{N}$$
    
Work in this notebook:

[1]  [Deep Sets](https://papers.nips.cc/paper/6931-deep-sets.pdf)

[2]  [Universal approximiations of invariant/equivariant functions by deep neural networks](https://arxiv.org/pdf/1903.01939.pdf)

# 1. Structural Invariance

**Kolmogorov–Arnold representation:**
Let  $ f : [0, 1]^M\rightarrow \mathbb{R}$ be an arbitrary multivariate
continuous function iff it has the representation 

$$f(x_1,...,x_M)=\rho\left(\sum_{m=1}^{M} \lambda_m(\phi(x_m)\right)$$


with continuous outer and inner functions $\rho: \mathbb{R}^{2M+1} \rightarrow \mathbb{R} $ and $\phi: \mathbb{R} \rightarrow \mathbb{R}^{2M+1}$. The inner function $\phi$ is independent of the function $f$.

**Deep Sets**: Represent $\rho$ and $\phi$ as feedforward neural networks. 

![DeepSets.png](attachment:d7d50434-2af8-466b-9ded-82df8b0e6a8c.png)

Deep Sets architecture as shown in [2]

# Application: Image Digit Sums
**Problem definition** Given a set of images of handwritten digits we want to compute the sum of all digits. This problem is permuation invariant since permuting the images will not affect the digit sum.
For simplicity we will first embed the MNIST dataset to a latent space such that the models can operate on vector data.

### Generate Data:

In [2]:
# Number of digits in the set
digit_length = 5

# Load the MNIST data from TF
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Use a simple model to train on MNIST to generate embeddings
embed_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64,name="Embedding"),
    tf.keras.layers.Dense(10)
])
embed_model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-2),  # Optimizer
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train on MNIST
embed_model.fit(x_train, y_train, epochs=5,validation_data=(x_test,y_test))

# Embed the data
embed_model = Model(embed_model.input, outputs=embed_model.get_layer("Embedding").output)
embed_train_x = embed_model(x_train).numpy()
embed_test_x = embed_model(x_test).numpy()


"""
    Takes the embedded MNIST data and returns a dataset for digit sum prediction. Each instance is a set of embedded instances with the label being the sum.
"""
def getDigitSum(digit_length,train_x,train_y,test_x,test_y):
    p = np.random.permutation(len(train_x))
    train_x = train_x[p]
    train_y = train_y[p]
    train_x = np.reshape(train_x,[-1,digit_length,64])
    train_y = np.reshape(train_y,[-1,digit_length])
    train_y = np.sum(train_y,1)

    p = np.random.permutation(len(test_x))
    test_x = test_x[p]
    test_y = test_y[p]
    test_x = np.reshape(test_x,[-1,digit_length,64])
    test_y = np.reshape(test_y,[-1,digit_length])
    test_y = np.sum(test_y,1)
    return train_x,train_y,test_x,test_y

11490434/11490434 [==============================] - 0s 0us/step


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/5
1875/1875 [==============================] - 18s 8ms/step - loss: 0.3884 - accuracy: 0.8880 - val_loss: 0.2313 - val_accuracy: 0.9307
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3119 - accuracy: 0.9171 - val_loss: 0.2314 - val_accuracy: 0.9384
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2809 - accuracy: 0.9279 - val_loss: 0.1832 - val_accuracy: 0.9544
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2703 - accuracy: 0.9322 - val_loss: 0.2577 - val_accuracy: 0.9482
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2676 - accuracy: 0.9334 - val_loss: 0.1787 - val_accuracy: 0.9628


In [3]:
# Generate data
train_x,train_y,test_x,test_y = getDigitSum(digit_length, embed_train_x, y_train, embed_test_x, y_test)

In [4]:
train_x.shape

(12000, 5, 64)

#### a)  Implement a simple Deep set model, and simple LSTM and Feedforward models as baseline (note that the Feedforward model just needs to flatten the training data from Batchsize X Set size X Feature to Batchsize X Feature before using dense layers

In [ ]:
# TODO Implement models - DeepSet
class DeepSet(tf.keras.layers.Layer):
    def __init__(self):
        super(DeepSet, self).__init__()
        # self.linear_1 = tf.keras.layers.Dense(32, activation='relu')
        #TODO: impl. deepset layers

    def call(self, inputs):
        # TODO: call dataset layers
        # x = self.linear_1(inputs)
        # x = tf.nn.relu(x)
        return self.linear_3(x)

SyntaxError: ignored

In [ ]:
# TODO Implement models - LSTM
class LSTM(tf.keras.layers.Layer):
    def __init__(self):
        super(LSTM, self).__init__()
        self.linear_1 = tf.keras.layers.DeepSet(32, activation='relu')
        self.linear_2 = tf.keras.layers.DeepSet(32)
        self.linear_3 = tf.keras.layers.DeepSet(32)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)

In [ ]:
# TODO Implement models - Dense
# note that the Feedforward model just needs to flatten the training data from Batchsize X Set size X Feature to Batchsize X Feature before using dense layers
class Dense(tf.keras.layers.Layer):
    def __init__(self):
        super(Dense, self).__init__()
        self.linear_1 = tf.keras.layers.Dense(32, activation='relu')
        self.linear_2 = tf.keras.layers.Dense(32)
        self.linear_3 = tf.keras.layers.Dense(32)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)

In [ ]:
# Initialize all three models
model_ds    = DeepSet()
model_lstm  = LSTM()
model_dense = Dense()

model_ds.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
    loss=tf.keras.losses.MeanAbsoluteError()
)

model_lstm.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
    loss=tf.keras.losses.MeanAbsoluteError()
)

model_dense.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.MeanAbsoluteError()
)

NameError: ignored

### Train models

In [ ]:
start = time.time()
hist_ds    = model_ds.fit(train_x, train_y, epochs=100, batch_size=128, shuffle=True,validation_data=(test_x,test_y),verbose=0)
print(f"Finished training deep set model in {time.time()-start}s")
start = time.time()
hist_lstm  = model_lstm.fit(train_x, train_y, epochs=100, batch_size=128, shuffle=True, validation_data=(test_x,test_y),verbose=0)
print(f"Finished training LSTM model in {time.time()-start}s")
start = time.time()
hist_dense = model_dense.fit(train_x, train_y, epochs=100, batch_size=128, shuffle=True, validation_data=(test_x,test_y),verbose=0)
print(f"Finished training Dense model in {time.time()-start}s")
print()
print(f"Deep Set model - Trainable parameters:  {np.sum([np.prod(i.shape) for i in model_ds.trainable_weights])}")
print(f"LSTM model     - Trainable parameters:  {np.sum([np.prod(i.shape) for i in model_lstm.trainable_weights])}")
print(f"Dense model    - Trainable parameters: {np.sum([np.prod(i.shape) for i in model_dense.trainable_weights])}")

Finished training deep set model in 90.26102614402771s
Finished training LSTM model in 134.73382329940796s
Finished training Dense model in 55.796677112579346s

Deep Set model - Trainable parameters:  52661
LSTM model     - Trainable parameters:  81361
Dense model    - Trainable parameters: 137089


#### b) Visualize the test loss of the three models and print the score

In [ ]:
# TODO

### Does the model generalize across different set sizes?

#### c) How does the model performance change with varying number of images in the set? Please evaluate the model trained on sets of length 5 by testing on sets of length 3 to 10. Plot the results in a graph

# 2. Structural Equivariant

The authors of [2] propose a permutation equivariant network by extending on the deep sets approach.
Compute deep sets for $n-1$ features and combine that output with the left out feature. Repeat over all features. Note that the embedding through $\phi$ can be computed once thus making the model computationally light.

![Equiv.png](attachment:bf6b2524-651d-4528-b1e1-6b06a4e9466e.png)

Equivariant architecture based on deep sets as proposed by [2]

# Application: Knapsack Problen
**Problem definition** Given a set of item with a corresponding weight and value we want to select a subset of these that maximizes the value while the combiend weight is lower than the maximum capacity. We can formulate this problem as a supervised machine learning problem by using a model that takes as input a set of weights and values for M items and outputs a binary decision for each item whether it is selected. This problem is permuation equivariant since permuting the order of items should only change the order of the predicted selection.

### Generate Data:

In [ ]:
num_items  = 8
train_size = 10000
val_size   = 10000

In [ ]:
# Generate a knapsack problem with "item_count" items which have a value between 1-99 and a weight between 0-0.6. 
# The optimal solution is binary vector of selected items for which the combined weight is smaller 1.0.
def knapsack_data(item_count=5):
    weights  = np.random.rand(item_count)*0.6
    values   = np.random.randint(1, 99, item_count)
    optimal  = brute_optimal(weights,values)
    return weights, values, optimal

# We can find an optimal solution for small problems by iterating all possible selections and selecting the one with highest value after pruning for weight.
def brute_optimal(weights,values):
    item_count = len(weights)
    lst = list(itertools.product([0, 1], repeat=item_count))
    
    total_value  = np.matmul(np.array(lst),values)
    total_weight = np.matmul(np.array(lst),weights)
    total_value[total_weight>1.0]=0
    
    return lst[np.argmax(total_value)]

# Generate a dataset with "train_size" and "val_size" knapsack problems with "num_items" items per problem
def get_data(train_size,val_size,num_items):
    start = time.time()
    X_weights = []
    X_values  = []
    Y = []

    for i in range(train_size):
        w,v,o = knapsack_data(num_items)
        X_weights.append(w)
        X_values.append(v)
        Y.append(o)
    X_weights = np.array(X_weights).astype(np.float32)
    X_values  = np.array(X_values).astype(np.float32)
    Y         = np.array(Y).astype(np.float32)

    X_weights_val = []
    X_values_val  = []
    Y_val = []

    for i in range(val_size):
        w,v,o = knapsack_data(num_items)
        X_weights_val.append(w)
        X_values_val.append(v)
        Y_val.append(o)
    X_weights_val = np.array(X_weights_val).astype(np.float32)
    X_values_val  = np.array(X_values_val).astype(np.float32)
    Y_val         = np.array(Y_val).astype(np.float32)
    print(f"{np.round(time.time()-start,2)}s")
    
    return X_weights, X_values, Y, X_weights_val, X_values_val, Y_val

In [ ]:
# Example Knapsack
knapsack_data(num_items)

(array([0.41339083, 0.49876553, 0.55634462, 0.04829673, 0.22248408,
        0.30891427, 0.01676307, 0.56470752]),
 array([52, 65, 72, 81, 55, 33, 54, 28]),
 (0, 0, 1, 1, 1, 0, 1, 0))

In [ ]:
# Generate dataset
X_weights, X_values, Y, X_weights_val, X_values_val, Y_val = get_data(train_size,val_size,num_items)

2.99s


### Equivariance

a) Implement the equivariant network proposed in [2] and then evaluate it on the Knapsack data provided. Compare against the dense model below.

In [ ]:
class Dense(Model):
    
    def __init__(self,item_count):
        
        super(Dense, self).__init__()
        
        self.d1   = tf.keras.layers.Dense(64,activation="relu")
        self.d2   = tf.keras.layers.Dense(64,activation="relu")
        self.out   = tf.keras.layers.Dense(item_count,activation="sigmoid")

    @tf.function
    def call(self, inputs):
        
        weights,values = inputs
        con = tf.keras.layers.concatenate([weights,values],axis=-1)
        
        d1  = self.d1(con)
        d2  = self.d2(d1)
        out = self.out(d2)
        
        return out

In [ ]:
# Initialize both models
model_dense = Dense(num_items)

model_dense.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Optimizer
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False)
)


model_equiv = EquiNet() #Todo

model_equiv.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Optimizer
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False)
)

### Train models

In [ ]:
start = time.time()
hist_dense = model_dense.fit([X_weights,X_values], Y, epochs=100, batch_size=128, shuffle=True,validation_data=([X_weights_val,X_values_val],Y_val),verbose=0)
print(f"Finished training Dense model in {time.time()-start}s")
start = time.time()
hist_equiv = model_equiv.fit([X_weights,X_values], Y, epochs=100, batch_size=128, shuffle=True,validation_data=([X_weights_val,X_values_val],Y_val),verbose=0)
print(f"Finished training Equivariant model in {time.time()-start}s")
print()
print(f"Equivariant model - Trainable parameters: {np.sum([np.prod(i.shape) for i in model_equiv.trainable_weights])}")
print(f"Dense model       - Trainable parameters: {np.sum([np.prod(i.shape) for i in model_dense.trainable_weights])}")

Finished training Dense model in 26.301701545715332s
Finished training Equivariant model in 56.54321050643921s

Equivariant model - Trainable parameters: 4385
Dense model       - Trainable parameters: 5768


### Visualize results

#### b) Print val results and plot val loss for both models

In [ ]:
# TODO

### Does the model generalize across different set sizes?

c) How does the model performance change with varying number of items? Evaluate the performance of the model trained on problems with 8 items on sets of length 4 to 12 and visualize the results